## SPECT/CT + Sestamibi Concordance and PPV Analysis

This section calculates:

- The number and percentage of Exact, Side, Partial, and Incorrect localizations for SPECT/CT + Sestamibi scans.
- The Positive Predictive Value (PPV), defined as:

\[
PPV = \frac{\text{True Positives (Exact + Side + Partial matches)}}{\text{True Positives + False Positives (Incorrect Localization)}}
\]

Only scans that localized (i.e. not Non-localizing or No Scan) are included.

The concordance function considers exact gland matches, partial matches (some but not all glands), and side matches (correct side but not exact gland).

In [2]:
import pandas as pd 
# preop_imaging_df = pd.read_excel('../../Data/side_concordance_results.xlsx')
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [3]:
def check_concordance(row):
    imaging = row['SPECT/CT-Sestamibi']
    surgical = row['Surgical Findings']

    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Clean imaging string
    imaging_clean = imaging.replace('SPECT/CT:', '').replace('Sestamibi:', '').replace('|', ',').strip()

    if imaging_clean == 'No SPECT/CT or Sestamibi Recorded':
        return 'Unknown'

    # Hyperplasia always Incorrect Localization
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    if imaging_clean == 'Non-localizing':
        return 'Non-localizing'

    imaging_glands = [g.strip().lower() for g in imaging_clean.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]

    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)

    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'
    elif imaging_set & surgical_set:
        return 'Yes (Partial)'

    # Side match check
    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        return 'Yes (Side)'

    return 'Incorrect Localization'

In [4]:
# Apply to dataframe
preop_imaging_df['SPECT/CT-Sestamibi Calculated Concordance'] = preop_imaging_df.apply(check_concordance, axis=1)

# Filter only localized scans (exclude Non-localizing and No Scan)
localized_df = preop_imaging_df[
    ~preop_imaging_df['SPECT/CT-Sestamibi'].isin(['No Scan', 'Non-localizing'])
]

# Calculate counts
concordance_col = 'SPECT/CT-Sestamibi Calculated Concordance'
total = localized_df.shape[0]
exact = (localized_df[concordance_col] == 'Yes (Exact)').sum()
side = (localized_df[concordance_col] == 'Yes (Side)').sum()
partial = (localized_df[concordance_col] == 'Yes (Partial)').sum()
incorrect = (localized_df[concordance_col] == 'Incorrect Localization').sum()


# Create summary table
summary_df = pd.DataFrame({
    'Match Type': ['Exact Match', 'Side Match', 'Partial Match', 'Incorrect Localization'],
    'Count': [exact, side, partial, incorrect],
    'Percentage': [f"{(c/total)*100:.2f}%" if total > 0 else 'N/A' for c in [exact, side, partial, incorrect]]
})

print(f" Total localized SPECT/CT + Sestamibi scans analyzed: {total}")
display(summary_df)


 Total localized SPECT/CT + Sestamibi scans analyzed: 299


,Match Type,Count,Percentage
0,Exact Match,117,39.13%
1,Side Match,137,45.82%
2,Partial Match,16,5.35%
3,Incorrect Localization,16,5.35%


In [7]:
# Apply to dataframe
preop_imaging_df['SPECT/CT-Sestamibi Calculated Concordance'] = preop_imaging_df.apply(check_concordance, axis=1)

# Filter only localized scans (exclude Non-localizing and No Scan)
localized_df = preop_imaging_df[
    ~preop_imaging_df['SPECT/CT-Sestamibi'].isin(['No Scan', 'Non-localizing'])
]

# Calculate counts directly from localized_df
concordance_col = 'SPECT/CT-Sestamibi Calculated Concordance'
exact = (localized_df[concordance_col] == 'Yes (Exact)').sum()
side = (localized_df[concordance_col] == 'Yes (Side)').sum()
partial = (localized_df[concordance_col] == 'Yes (Partial)').sum()
incorrect = (localized_df[concordance_col] == 'Incorrect Localization').sum()

# Calculate PPV
true_positives = exact + side + partial
false_positives = incorrect

ppv = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else None

print(f"PPV for localized SPECT/CT + Sestamibi scans: {ppv:.2%}" if ppv is not None else "PPV cannot be calculated.")

PPV for localized SPECT/CT + Sestamibi scans: 94.41%
